In [ ]:
# Prosjektoppgave: Norgespris på strøm – lønner det seg?

## Introduksjon
Regjeringen foreslår en **Norgespris** på strøm med fastpris 40 øre/kWh. I dette prosjektet skal du undersøke om dette lønner seg i ulike deler av landet, basert på historiske strømpriser og enkle prediksjoner.

Dere skal jobbe i grupper og bruke Python (Pandas, Matplotlib/Seaborn, Scikit-learn) for å hente inn, analysere og visualisere data. Prosjektet går over to dager og deles i flere deler.

---

## Del 1: Hente og utforske data
- Hent strømpriser for de siste 2–3 årene (områder NO1–NO5).  
- Lag en dataframe med: dato, time, pris (øre/kWh), område.  
- Rens data (manglende verdier, datatype, outliers).

**Spørsmål:**  
- Hvordan ser datasettet ut?  
Jeg har kopiert data fra fortum.com for strøm priser i forskjellige områder i Norge for 2022, 2023, 2024 og limt inn i et nytt excel ark for å ha oversikt og analysere. Her har jeg brukt nemlig gjennomsnittlig spotpris per måned og til sammen for 3 år (36 måneder) og 5 områder. Det vil si fra januar 2022 til desember 2024, 180 rader til sammen.



- Hvor mye varierer prisene mellom ulike områder?  
Prisene varierer svært mye mellom områdene, og dette er det mest markante trekket i datasettet. Variasjonen er størst i Sør-Norge (NO1, NO2, NO5)sammenlignet med Midt- og Nord-Norge(NO3, NO4), og varierer også sterkt etter årstid og år. 

Sør-Norge (NO1, NO2, NO5): Disse områdene har de høyeste gjennomsnittsprisene, hvor NO2 er dyrest. Den gjennomsnittlige prisen er rundt 5–6 ganger høyere enn i Nord-Norge.Midt- og Nord-Norge (NO3, NO4): Disse områdene har betydelig lavere og mindre variable priser. NO4 er det klart billigste området, ofte med priser under 0.30 øre/kWh i gjennomsnitt.

For eksempel August 2022
Sør NO2:5.431 øre/kWhNord NO4:0.028 øre/kWh.
Dette tilsvarer at prisen i Sør-Norge var 194 ganger høyere enn i Nord-Norge i denne måneden.
---

## Del 2: Visualisering av strømpriser
Lag minst **3–4 forskjellige visualiseringer** og forklar hvorfor den formen passer.

Forslag:  
1. Linjediagram: Pris over tid (dag/måned/år).  
2. Boksplott: Fordeling av priser per måned eller område.  
3. Histogram: Hvilke prisnivåer er vanligst?  
4. Heatmap: Pris per time på døgnet vs måned.  

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. DATALASTING OG FORBEREDELSE (TILPASSET ENGELSKE KOLONNER OG D/M/Å) ---

# Definer filstien
file_path = r'C:\Users\bayez\OneDrive\Masaüstü\kongsberg\Introduksjon-til-dataanalyse\datasett\dataset-norgespris.xlsx'

# Sjekk om filen eksisterer
if not os.path.exists(file_path):
    print(f"FEIL: Filen ble ikke funnet på stien: {file_path}")
    exit()

# Last inn Excel-dataen til en Pandas DataFrame
try:
    df = pd.read_excel(file_path)
    print("Dataen lastet inn. Starter forberedelse...")

    # 1. Gi kolonnene norske navn og sjekk at de engelske navnene finnes
    df.rename(columns={
        'Month': 'Dato',
        'Price (øre/kWh)': 'Pris (øre/kWh)',
        'Geographical Area': 'Område'
    }, inplace=True)

    # Rensing 2: Konverter 'Dato' til datetime. 
    # Bruker 'dayfirst=True' for å sikre korrekt D/M/Å-tolkning.
    df['Dato'] = pd.to_datetime(df['Dato'], dayfirst=True, errors='coerce')
    
    # Rensing 3: Sikre numerisk 'Pris' og fjerne ugyldige rader
    df['Pris (øre/kWh)'] = pd.to_numeric(df['Pris (øre/kWh)'], errors='coerce')
    df.dropna(subset=['Dato', 'Pris (øre/kWh)', 'Område'], inplace=True) 
    
    # 4. Trekk ut tidsvariabler som trengs for visualisering
    df['Måned_Nummer'] = df['Dato'].dt.month
  
    
    print(f"Dataforberedelse fullført. Antall rader: {len(df)}")

except KeyError:
    print("\nFEIL: Sjekk de engelske kolonnenavnene!")
    print("Koden forventer nøyaktig: 'Month', 'Price (øre/kWh)', og 'Geographical Area'.")
    print(f"Kolonnene i din DataFrame er: {df.columns.tolist()}")
    exit()
except Exception as e:
    print(f"En ukjent feil oppstod under innlasting/forberedelse: {e}")
    exit()

# --- 2. VISUALISERINGER ---

# ----------------------------------------------------------------------
## 1. Linjediagram: Pris over Tid (Daglig Gjennomsnitt)
# ----------------------------------------------------------------------
# Aggreger til daglig gjennomsnitt
df_daily = df.groupby(['Område', df['Dato'].dt.date])['Pris (øre/kWh)'].mean().reset_index()
df_daily['Dato'] = pd.to_datetime(df_daily['Dato'])

plt.figure(figsize=(14, 6))
sns.lineplot(data=df_daily, x='Dato', y='Pris (øre/kWh)', hue='Område', errorbar=None)
plt.title('1. Gjennomsnittlig Daglig Strømprisutvikling etter Område', fontsize=16)
plt.xlabel('Dato', fontsize=12)
plt.ylabel('Pris (øre/kWh)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='Prisområde')
plt.show()

# ----------------------------------------------------------------------
## 2. Boksplott: Fordeling av Priser per Område
# ----------------------------------------------------------------------
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Område', y='Pris (øre/kWh)', palette='viridis')
plt.title('2. Timeprisfordeling og Variabilitet per Prisområde', fontsize=16)
plt.xlabel('Prisområde', fontsize=12)
plt.ylabel('Pris (øre/kWh)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

# ----------------------------------------------------------------------
## 3. Histogram: Hvilke Prisnivåer er Vanligst?
# ----------------------------------------------------------------------
plt.figure(figsize=(10, 6))
sns.histplot(df['Pris (øre/kWh)'], bins=50, kde=True, color='skyblue', edgecolor='black')
plt.title('3. Frekvensfordeling av Timepriser (Alle Områder og Tid)', fontsize=16)
plt.xlabel('Pris (øre/kWh)', fontsize=12)
plt.ylabel('Frekvens (Antall Timer)', fontsize=12)
plt.axvline(df['Pris (øre/kWh)'].mean(), color='red', linestyle='--', label=f"Gjennomsnitt: {df['Pris (øre/kWh)'].mean():.2f}")
plt.legend()
plt.show()




**Spørsmål:**  
- Når på året er prisene høyest/lavest?  
Høyest (Vinter): Desember, Januar og Februar. Kaldt vær øker strømforbruket for oppvarming, samtidig som lavere tilsig til vannmagasinene og høyere etterspørsel i Europa presser prisene opp.
Lavest (Sommer): Mai, Juni og Juli. Varmt vær reduserer behovet for oppvarming, og snøsmelting gir økt vannkraftproduksjon og høyt fyllingsnivå i vannmagasinene.
- Hvordan varierer prisen gjennom døgnet? 
Jeg fant dessverre ikke detaljert datasett for pris på døgnet. Men data viser variasjon mellom månedene og pris forskjeller mellom områder. 
- Hvilke forskjeller ser vi mellom områder?  
Forskjellene mellom prisområdene er den mest markante strukturelle forskjellen i norske strømpriser. Sør-Norge (NO1, NO2, NO5): Tett koblet til det europeiske markedet via kabler. Prisene svinger mye og er ofte ekstremt høye under tørke eller høye gasspriser i Europa.Midt-Norge (NO3):Er ofte priset uavhengig av Sør-Norge, men kan ha perioder med høye priser dersom overføringskapasiteten til Nord-Norge er strupet.
Nord-Norge (NO4): Svært høy kraftproduksjon i forhold til forbruk, lite koblet til det europeiske markedet, noe som holder prisene stabile og lave.
---

## Del 3: Brukerfunksjoner med postnummer
Lag funksjoner som tar inn **postnummer** → finner hvilket NO-område det tilhører → returnerer:  
- Dagens strømpriser (time for time).  
- Snittpris siste 3 år.  
- Gjennomsnittspris per time (når på døgnet er strømmen dyrest/billigst?).  
- Lag en rapport med grafer for dette området.

---

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
from datetime import datetime

# ==============================================================================
# 1. KONFIGURASJON OG OPPSLAGSTABELL
# ==============================================================================

# Definer filstien
file_path = r'C:\Users\bayez\OneDrive\Masaüstü\kongsberg\Introduksjon-til-dataanalyse\datasett\dataset-norgespris.xlsx'

# Simulerer oppslagstabell for postnummer -> NO-område
# OBS: Dette er en SVÆRT FORENLET versjon. MÅ erstattes med en komplett oversikt.
NO_OMRÅDE_MAPPING = {
    '0101': 'Øst NO1',  # Oslo/Østlandet
    '5003': 'Vest NO5',  # Bergen/Vestlandet (Setter denne som standard test)
    '7001': 'Midt NO3',  # Trondheim/Trøndelag
    '9001': 'Nord NO4',  # Tromsø/Nord-Norge
    '4600': 'Sør NO2',  # Kristiansand/Sørlandet
    '3000': 'Øst NO1',
    '5500': 'Vest NO5',
    '8000': 'Nord NO4',
}

def hent_no_område(postnummer):
    """Sjekker postnummeret mot oppslagstabellen for å finne NO-området."""
    postnummer_str = str(postnummer).zfill(4)
    område = NO_OMRÅDE_MAPPING.get(postnummer_str[:4])
    return område

# ==============================================================================
# 2. DATALASTING OG RENSING
# ==============================================================================

def last_og_rens_data(file_path):
    """Laster inn Excel-filen, renser data, og konverterer kolonner til norsk."""
    if not os.path.exists(file_path):
        print(f"FEIL: Filen ble ikke funnet på stien: {file_path}")
        return None
    
    try:
        df = pd.read_excel(file_path)
        print("Data lastet inn. Starter rensing...")

        # Gi kolonnene norske navn og strip kolonnetekst (fjerner mellomrom)
        df.columns = df.columns.str.strip() 
        df.rename(columns={
            'Month': 'Dato',
            'Price (øre/kWh)': 'Pris (øre/kWh)',
            'Geographical Area': 'Område'
        }, inplace=True)
        
        # 1. Konverter 'Dato' med D/M/Å-prioritet
        df['Dato'] = pd.to_datetime(df['Dato'], dayfirst=True, errors='coerce')
        
        # 2. Sikre numerisk 'Pris'
        df['Pris (øre/kWh)'] = pd.to_numeric(df['Pris (øre/kWh)'], errors='coerce')
        
        # 3. Fjern rader med ugyldig dato eller pris
        df.dropna(subset=['Dato', 'Pris (øre/kWh)', 'Område'], inplace=True) 
        
        # 4. Trekk ut tidsvariabler
        df['Måned_Nummer'] = df['Dato'].dt.month
        df['Time_på_Døgnet'] = df['Dato'].dt.hour
        
        print(f"Dataforberedelse fullført. Antall rader: {len(df)}")
        return df

    except KeyError as e:
        print(f"\nFEIL: Klarte ikke å finne forventede engelske kolonner. Sjekk at navnet '{e}' er riktig.")
        print(f"Kolonnene i din DataFrame er: {df.columns.tolist()}")
        return None
    except Exception as e:
        print(f"En ukjent feil oppstod under innlasting/forberedelse: {e}")
        return None

# ==============================================================================
# 3. ANALYSE- OG RAPPORTERINGSFUNKSJONER
# ==============================================================================

def beregn_priser_for_område(postnummer, df):
    """
    Henter NO-område og beregner nøkkelpriser for området.
    """
    område = hent_no_område(postnummer)
    
    if område is None:
        return {"error": f"Ukjent område for postnummer {postnummer}. Sjekk mappingen."}, None

    # Ingen bruk av 'data' her. Bruker 'df'
    df_omraade = df[df['Område'] == område].copy() 
    
    if df_omraade.empty:
        return {"error": f"Ingen data funnet for område {område}."}, None

    # --- Snittpris siste 3 år ---
    snittpris_3aar = df_omraade['Pris (øre/kWh)'].mean()

    # --- Gjennomsnittspris per time (døgnvariasjon) ---
    snittpris_time = df_omraade.groupby('Time_på_Døgnet')['Pris (øre/kWh)'].mean().reset_index()
    dyrest_time = snittpris_time.loc[snittpris_time['Pris (øre/kWh)'].idxmax()]
    billigst_time = snittpris_time.loc[snittpris_time['Pris (øre/kWh)'].idxmin()]

    # --- Dagens strømpriser ---
    i_dag = dt.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    df_idag = df_omraade[df_omraade['Dato'].dt.normalize() == i_dag]
    
    # Fallback til siste tilgjengelige dag hvis i dag mangler
    if df_idag.empty and not df_omraade.empty:
        siste_dag_data = df_omraade['Dato'].dt.normalize().max()
        df_idag = df_omraade[df_omraade['Dato'].dt.normalize() == siste_dag_data]
        dato_rapport = siste_dag_data.strftime('%d.%m.%Y')
    else:
        dato_rapport = i_dag.strftime('%d.%m.%Y')
    
    pris_idag_rapport = df_idag.sort_values('Dato').set_index('Dato')['Pris (øre/kWh)'].to_dict()

    return {
        "område": område,
        "snittpris_3aar": snittpris_3aar,
        "dyrest_time": f"Kl. {int(dyrest_time['Time_på_Døgnet']):02d}", 
        "billigst_time": f"Kl. {int(billigst_time['Time_på_Døgnet']):02d}",
        "pris_idag_rapport": pris_idag_rapport,
        "dato_rapport": dato_rapport
    }, df_omraade


def lag_rapport_grafer(df_omraade, analyse_data):
    """
    Lager en visuell rapport for det valgte prisområdet.
    """
    område = analyse_data['område']
    
    # ------------------------------------------
    # Graf 1: Gjennomsnittspris per time på døgnet
    # ------------------------------------------
    plt.figure(figsize=(10, 6))
    snitt_time = df_omraade.groupby('Time_på_Døgnet')['Pris (øre/kWh)'].mean().reset_index()
    sns.barplot(x='Time_på_Døgnet', y='Pris (øre/kWh)', data=snitt_time, color='#1f77b4') # Ingen bruk av 'data' her.
    plt.axvline(x=snitt_time['Time_på_Døgnet'][snitt_time['Pris (øre/kWh)'].idxmax()], color='red', linestyle='--', label=f"Dyrest: {analyse_data['dyrest_time']}")
    plt.axvline(x=snitt_time['Time_på_Døgnet'][snitt_time['Pris (øre/kWh)'].idxmin()], color='green', linestyle='--', label=f"Billigst: {analyse_data['billigst_time']}")
    plt.title(f'Gjennomsnittlig Timepris gjennom døgnet for {område}', fontsize=14)
    plt.xlabel('Time på Døgnet (0-23)')
    plt.ylabel('Snittpris (øre/kWh)')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.show()
    
    # ------------------------------------------
    # Graf 2: Prisutvikling over hele perioden (Månedlig trend)
    # ------------------------------------------
    plt.figure(figsize=(14, 6))
    månedlig_snitt = df_omraade.groupby(df_omraade['Dato'].dt.to_period('M'))['Pris (øre/kWh)'].mean().reset_index()
    månedlig_snitt['Dato'] = månedlig_snitt['Dato'].astype(str)
    
    sns.lineplot(data=månedlig_snitt, x='Dato', y='Pris (øre/kWh)', marker='o') # Ingen bruk av 'data' her.
    plt.title(f'Månedlig Snittpris over tid for {område}', fontsize=14)
    plt.xlabel('Måned')
    plt.ylabel('Snittpris (øre/kWh)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# ==============================================================================
# 4. HOVEDKONTROLL OG BRUK
# ==============================================================================

# Last inn data
df_full = last_og_rens_data(file_path)

if df_full is not None:
    # --- Sett postnummeret du vil analysere her ---
    TEST_POSTNUMMER = 5003 
    
    # Kjør analyse. df_full er korrekt variabelnavn her.
    analyse_resultat, df_spesifikt_område = beregn_priser_for_område(TEST_POSTNUMMER, df_full)
    
    if 'error' in analyse_resultat:
        print(f"\nFeil ved analyse: {analyse_resultat['error']}")
    else:
        # Skriver ut rapporten i tekstformat
        print("\n" + "="*50)
        print(f"STRØMPRISRAPPORT FOR {analyse_resultat['område']} (Postnr: {TEST_POSTNUMMER})")
        print("="*50)
        print(f"1. Snittpris (Hele perioden): {analyse_resultat['snittpris_3aar']:.2f} øre/kWh")
        print(f"2. Dyrest time i snitt: {analyse_resultat['dyrest_time']}")
        print(f"3. Billigst time i snitt: {analyse_resultat['billigst_time']}")
        
        print(f"\nTimepriser for dato {analyse_resultat['dato_rapport']}:")
        for time, pris in analyse_resultat['pris_idag_rapport'].items():
            print(f"  {time.strftime('%H:%M')}: {pris:.3f} øre/kWh")
        
        # Lager grafisk rapport
        lag_rapport_grafer(df_spesifikt_område, analyse_resultat)


## Del 4: Prediksjonsmodell

Bygg en enkel modell som estimerer **forventet gjennomsnittspris framover**.  
Start med månedlige gjennomsnittspriser og utvid deretter til døgn- og timepriser.  

- **Input:** måned, område, evt. årstrend.  
- **Modell:** enkel lineær regresjon eller sesongmønster.  
- **Output:** predikert snittpris neste måned(er).  

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
from datetime import datetime
import statsmodels.api as sm 
from statsmodels.regression.linear_model import OLS 

# ==============================================================================
# 1. KONFIGURASJON OG OPPSLAGSTABELL
# ==============================================================================

# Definer filstien til Excel-filen din.
file_path = r'C:\Users\bayez\OneDrive\Masaüstü\kongsberg\Introduksjon-til-dataanalyse\datasett\dataset-norgespris.xlsx'

# Simulerer oppslagstabell for postnummer -> NO-område
# OBS: Dette er en SVÆRT FORENLET versjon. MÅ erstattes med en komplett oversikt.
NO_OMRÅDE_MAPPING = {
    '0101': 'Øst NO1',  # Oslo/Østlandet
    '5003': 'Vest NO5',  # Bergen/Vestlandet
    '7001': 'Midt NO3',  # Trondheim/Trøndelag
    '9001': 'Nord NO4',  # Tromsø/Nord-Norge
    '4600': 'Sør NO2',  # Kristiansand/Sørlandet
    '3000': 'Øst NO1',
    '5500': 'Vest NO5',
    '8000': 'Nord NO4',
}

# Referansepris for å vurdere lønnsomhet av Norgespris (øre/kWh)
REFERANSEPRIS_NKR = 40.0 

def hent_no_område(postnummer):
    """Sjekker postnummeret mot oppslagstabellen for å finne NO-området."""
    postnummer_str = str(postnummer).zfill(4)
    område = NO_OMRÅDE_MAPPING.get(postnummer_str[:4])
    return område

# ==============================================================================
# 2. DATALASTING OG RENSING
# ==============================================================================

def last_og_rens_data(file_path):
    """Laster inn Excel-filen, renser data, og konverterer kolonner til norsk."""
    if not os.path.exists(file_path):
        print(f"FEIL: Filen ble ikke funnet på stien: {file_path}")
        return None
    
    try:
        df = pd.read_excel(file_path)
        print("Data lastet inn. Starter rensing...")

        # Gi kolonnene norske navn og strip kolonnetekst (fjerner mellomrom)
        df.columns = df.columns.str.strip() 
        df.rename(columns={
            'Month': 'Dato',
            'Price (øre/kWh)': 'Pris (øre/kWh)',
            'Geographical Area': 'Område'
        }, inplace=True)
        
        # 1. Konverter 'Dato' med D/M/Å-prioritet
        df['Dato'] = pd.to_datetime(df['Dato'], dayfirst=True, errors='coerce')
        
        # 2. Sikre numerisk 'Pris'
        df['Pris (øre/kWh)'] = pd.to_numeric(df['Pris (øre/kWh)'], errors='coerce')
        
        # 3. Fjern rader med ugyldig dato eller pris
        df.dropna(subset=['Dato', 'Pris (øre/kWh)', 'Område'], inplace=True) 
        
        # 4. Trekk ut tidsvariabler
        df['Måned_Nummer'] = df['Dato'].dt.month
        df['Time_på_Døgnet'] = df['Dato'].dt.hour
        
        print(f"Dataforberedelse fullført. Antall rader: {len(df)}")
        return df

    except KeyError as e:
        print(f"\nFEIL: Klarte ikke å finne forventede engelske kolonner. Sjekk at navnet '{e}' er riktig.")
        print(f"Kolonnene i din DataFrame er: {df.columns.tolist()}")
        return None
    except Exception as e:
        print(f"En ukjent feil oppstod under innlasting/forberedelse: {e}")
        return None

# ==============================================================================
# 3. ANALYSEFUNKSJONER
# ==============================================================================

def beregn_priser_for_område(postnummer, df):
    """
    Henter NO-område og beregner nøkkelpriser for området.
    """
    område = hent_no_område(postnummer)
    
    if område is None:
        return {"error": f"Ukjent område for postnummer {postnummer}. Sjekk mappingen."}, None

    df_omraade = df[df['Område'] == område].copy() 
    
    if df_omraade.empty:
        return {"error": f"Ingen data funnet for område {område}."}, None

    # --- Snittpris siste 3 år ---
    snittpris_3aar = df_omraade['Pris (øre/kWh)'].mean()

    # --- Gjennomsnittspris per time (døgnvariasjon) ---
    snittpris_time = df_omraade.groupby('Time_på_Døgnet')['Pris (øre/kWh)'].mean().reset_index()
    dyrest_time = snittpris_time.loc[snittpris_time['Pris (øre/kWh)'].idxmax()]
    billigst_time = snittpris_time.loc[snittpris_time['Pris (øre/kWh)'].idxmin()]

    # --- Dagens strømpriser ---
    # Bruker dt.datetime for å unngå Attribute Error, setter tid til midnatt
    i_dag = dt.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    df_idag = df_omraade[df_omraade['Dato'].dt.normalize() == i_dag]
    
    # Fallback til siste tilgjengelige dag hvis i dag mangler
    if df_idag.empty and not df_omraade.empty:
        siste_dag_data = df_omraade['Dato'].dt.normalize().max()
        df_idag = df_omraade[df_omraade['Dato'].dt.normalize() == siste_dag_data]
        dato_rapport = siste_dag_data.strftime('%d.%m.%Y')
    else:
        dato_rapport = i_dag.strftime('%d.%m.%Y')
    
    pris_idag_rapport = df_idag.sort_values('Dato').set_index('Dato')['Pris (øre/kWh)'].to_dict()

    return {
        "område": område,
        "snittpris_3aar": snittpris_3aar,
        # Konverterer til int for å sikre at '02d' formatkoden fungerer
        "dyrest_time": f"Kl. {int(dyrest_time['Time_på_Døgnet']):02d}", 
        "billigst_time": f"Kl. {int(billigst_time['Time_på_Døgnet']):02d}",
        "pris_idag_rapport": pris_idag_rapport,
        "dato_rapport": dato_rapport
    }, df_omraade

# ==============================================================================
# 4. MODELLERING OG PREDIKSJON
# ==============================================================================

def estimer_fremtidig_pris(df_omraade, antall_måneder=6):
    """
    Bygger en enkel tidsseriemodell (Lineær trend + Sesong) 
    og predikerer gjennomsnittlig pris for de neste månedene.
    """
    if df_omraade.empty:
        return None, "Ingen data for estimering."

    # 1. Aggreger til månedlig snitt
    df_månedlig = df_omraade.groupby(
        df_omraade['Dato'].dt.to_period('M')
    )['Pris (øre/kWh)'].mean().reset_index()

    df_månedlig['Dato'] = df_månedlig['Dato'].dt.to_timestamp()
    
    # Sorterer data for å sikre at Tid-variabelen er korrekt (viktig!)
    df_månedlig.sort_values('Dato', inplace=True) 
    
    if len(df_månedlig) < 14:
        return None, f"For få månedlige datapunkter ({len(df_månedlig)}). Krever minst 14 måneder."
    
    # 2. Lag variabler for Lineær regresjon
    df_månedlig['Tid'] = (df_månedlig['Dato'] - df_månedlig['Dato'].min()).dt.days / 30.44 
    df_månedlig['Måned'] = df_månedlig['Dato'].dt.month
    
    # Sesongvariabel (Dummy-variabler)
    sesong_dummies = pd.get_dummies(df_månedlig['Måned'], prefix='Måned', drop_first=True)
    
    # Lag modellens input
    X = pd.concat([df_månedlig['Tid'], sesong_dummies], axis=1)
    y = df_månedlig['Pris (øre/kWh)']

    # 3. Klargjøring av treningsdata (X_clean og y_clean)
    
    # TVING NUMERISK og legg til konstant
    X_med_konstant = sm.add_constant(X.apply(pd.to_numeric, errors='coerce'))
    y_numerisk = pd.to_numeric(y, errors='coerce')

    # Fjerner rader med NaN som kan ha oppstått i konverteringen
    mask = X_med_konstant.isna().any(axis=1) | y_numerisk.isna()
    X_clean_df = X_med_konstant[~mask]
    y_clean_df = y_numerisk[~mask]
    
    # !!! NØKKELKORRIGERING: Konverterer til NumPy array av type float før OLS !!!
    X_clean = X_clean_df.values.astype(float)
    y_clean = y_clean_df.values.astype(float)
    
    # 4. Tren den Lineære regresjonsmodellen (OLS)
    try:
        modell = OLS(y_clean, X_clean).fit()
    except ValueError as e:
         # Skriver ut kolonnene for feilsøking
         return None, f"Modellering feilet: {e}. Sjekk kolonnene: {X_clean_df.columns.tolist()}"
    
    # 5. Lag prediksjonsramme for fremtidige måneder
    siste_dato = df_månedlig['Dato'].max()
    prediksjonsdatoer = pd.date_range(start=siste_dato, periods=antall_måneder + 1, freq='MS')[1:]
    
    X_prediksjon = pd.DataFrame()
    X_prediksjon['Dato'] = prediksjonsdatoer
    X_prediksjon['Tid'] = (X_prediksjon['Dato'] - df_månedlig['Dato'].min()).dt.days / 30.44
    X_prediksjon['Måned'] = prediksjonsdatoer.month
    
    sesong_prediksjon = pd.get_dummies(X_prediksjon['Måned'], prefix='Måned')
    
    # 6. Sett sammen prediksjonsmatrisen
    X_prediksjon_final_df = pd.DataFrame(index=X_prediksjon.index)
    
    # Bruker kolonnene fra den rensede treningsdataen for å sikre perfekt match
    kolonner_uten_konstant = X_clean_df.drop(columns='const').columns

    for col in kolonner_uten_konstant:
        if col == 'Tid':
            X_prediksjon_final_df[col] = X_prediksjon['Tid']
        elif col in sesong_prediksjon.columns:
            X_prediksjon_final_df[col] = sesong_prediksjon[col]
        else:
            X_prediksjon_final_df[col] = 0 # Referansemåned eller manglende måned
            
    # TVING NUMERISK og legg til konstant
    X_prediksjon_final_df = X_prediksjon_final_df.apply(pd.to_numeric, errors='coerce')
    X_prediksjon_final_df = sm.add_constant(X_prediksjon_final_df, has_constant='add')
    
    # !!! NØKKELKORRIGERING: Konverterer til NumPy array av type float før prediksjon !!!
    X_prediksjon_final = X_prediksjon_final_df.values.astype(float)
    
    # 7. Prediker priser
    predikerte_priser = modell.predict(X_prediksjon_final)
    
    resultat = pd.DataFrame({
        'Måned': prediksjonsdatoer.strftime('%Y-%m'),
        'Predikert Pris (øre/kWh)': predikerte_priser
    }).reset_index(drop=True)
    
    return resultat, None

# ==============================================================================
# 5. RAPPORTERINGSFUNKSJONER (GRAFER)
# ==============================================================================

def lag_rapport_grafer(df_omraade, analyse_data, df_estimater=None):
    """
    Lager en visuell rapport for det valgte prisområdet, inkludert estimater.
    """
    område = analyse_data['område']
    
    # ------------------------------------------
    # Graf 1: Gjennomsnittspris per time på døgnet (Døgnvariasjon)
    # ------------------------------------------
    plt.figure(figsize=(10, 6))
    snitt_time = df_omraade.groupby('Time_på_Døgnet')['Pris (øre/kWh)'].mean().reset_index()
    sns.barplot(x='Time_på_Døgnet', y='Pris (øre/kWh)', data=snitt_time, color='#1f77b4') 
    plt.axvline(x=snitt_time['Time_på_Døgnet'][snitt_time['Pris (øre/kWh)'].idxmax()], color='red', linestyle='--', label=f"Dyrest: {analyse_data['dyrest_time']}")
    plt.axvline(x=snitt_time['Time_på_Døgnet'][snitt_time['Pris (øre/kWh)'].idxmin()], color='green', linestyle='--', label=f"Billigst: {analyse_data['billigst_time']}")
    plt.title(f'Gjennomsnittlig Timepris gjennom døgnet for {område}', fontsize=14)
    plt.xlabel('Time på Døgnet (0-23)')
    plt.ylabel('Snittpris (øre/kWh)')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.show()
    
    # ------------------------------------------
    # Graf 2: Prisutvikling og Estimat (Månedlig trend)
    # ------------------------------------------
    plt.figure(figsize=(14, 6))
    månedlig_snitt = df_omraade.groupby(df_omraade['Dato'].dt.to_period('M'))['Pris (øre/kWh)'].mean().reset_index()
    månedlig_snitt['Dato'] = månedlig_snitt['Dato'].astype(str)
    
    sns.lineplot(data=månedlig_snitt, x='Dato', y='Pris (øre/kWh)', marker='o', label='Historisk Snittpris')
    
    if df_estimater is not None:
        sns.lineplot(data=df_estimater, x='Måned', y='Predikert Pris (øre/kWh)', marker='x', linestyle='--', color='red', label='Predikert Snittpris')
        plt.axhline(REFERANSEPRIS_NKR, color='orange', linestyle='-', label=f'Norgespris Grense ({REFERANSEPRIS_NKR:.0f} øre/kWh)')

    plt.title(f'Historisk og Estimerte Månedlige Priser for {område}', fontsize=14)
    plt.xlabel('Måned')
    plt.ylabel('Snittpris (øre/kWh)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# ==============================================================================
# 6. HOVEDKONTROLL OG BRUK
# ==============================================================================

# Last inn data
df_full = last_og_rens_data(file_path)

if df_full is not None:
    # --- Sett postnummeret du vil analysere her ---
    TEST_POSTNUMMER = 5003 
    ANTALL_MÅNEDER = 6 # Hvor mange måneder fram i tid skal estimeres
    
    # 1. Kjør detaljert analyse
    analyse_resultat, df_spesifikt_område = beregn_priser_for_område(TEST_POSTNUMMER, df_full)
    
    # 2. Kjør prediksjonsmodell
    df_estimater, feilmelding = estimer_fremtidig_pris(df_spesifikt_område, ANTALL_MÅNEDER)
    
    if 'error' in analyse_resultat:
        print(f"\nFeil ved analyse: {analyse_resultat['error']}")
    else:
        # Skriver ut rapporten i tekstformat
        print("\n" + "="*70)
        print(f"STRØMPRISRAPPORT FOR {analyse_resultat['område']} (Postnr: {TEST_POSTNUMMER})")
        print("="*70)
        print(f"1. Snittpris (Hele perioden): {analyse_resultat['snittpris_3aar']:.2f} øre/kWh")
        print(f"2. Dyrest time i snitt: {analyse_resultat['dyrest_time']}")
        print(f"3. Billigst time i snitt: {analyse_resultat['billigst_time']}")
        print("="*70)
        
        # 3. Presenter estimater og lønnsomhet
        
        if feilmelding:
            print(f"\nADVARSEL: Estimeringsmodell feilet: {feilmelding}")
        else:
            print(f"\nESTIMERT PRISUTVIKLING DE NESTE {ANTALL_MÅNEDER} MÅNEDENE:")
            
            # Sammenligning med Norgespris (Lønnsomt = > REFERANSEPRIS_NKR)
            df_estimater['Lønnsomhet'] = df_estimater['Predikert Pris (øre/kWh)'].apply(
                lambda p: 'Lønnsomt (over 40 øre)' if p >= REFERANSEPRIS_NKR else 'Ulønnsomt (under 40 øre)'
            )
            
            # Utskrift av estimatene
            print(df_estimater.to_string(index=False))
            
            print("\n" + "—"*70)
            
            # Svar på Spørsmål 1: Lønner det seg med Norgespris (40 øre/kWh)?
            lønnsomme_perioder = df_estimater[df_estimater['Lønnsomhet'] == 'Lønnsomt (over 40 øre)']
            
            if lønnsomme_perioder.empty:
                print(f"SVAR: Basert på estimatet forventes det **ulønnsomt** å bruke Norgespris (40 øre/kWh) de neste {ANTALL_MÅNEDER} månedene.")
            else:
                print(f"SVAR: Norgespris (40 øre/kWh) **kan lønne seg** i de månedene der gjennomsnittsprisen estimeres å overstige {REFERANSEPRIS_NKR} øre/kWh.")
                
            # Svar på Spørsmål 2: Hvilke perioder vil det mest sannsynlig være lønnsomt/ulønnsomt?
            print("\nPerioder for lønnsomhet (vs. 40 øre/kWh):")
            print(df_estimater.groupby('Lønnsomhet')['Måned'].apply(lambda x: ', '.join(x)).reset_index().to_string(index=False))

        # 4. Lager grafisk rapport (inkluderer nå prediksjonsgraf)
        lag_rapport_grafer(df_spesifikt_område, analyse_resultat, df_estimater)
        
**Spørsmål:**  
- Lønner det seg med Norgespris (40 øre/kWh) i mitt område?  
- Hvilke perioder vil det mest sannsynlig være lønnsomt/ulønnsomt?  

### Ekstraoppgave: Prediksjoner på flere tidsskalaer
Prøv å lage modeller på ulike nivåer:  
1. **Månedlige priser** – Estimer gjennomsnitt per måned framover.  
2. **Døgnpriser** – Prediker gjennomsnittspris per dag for neste uke/måned.  
3. **Timepriser** – Lag en enkel modell som predikerer time-for-time pris for et døgn.  

Sammenlign minst to ulike modeller (f.eks. lineær regresjon, logistisk regresjon eller gjennomsnitt av historiske mønstre).  

**Diskusjon:**  
- Hvilke modeller passer best for månedlige trender? 
For å modellere og forutsi månedlige trender i strømpriser, som typisk inneholder både langsiktig utvikling og årlig syklisitet, er følgende modeller best egnet:

SARIMA (Seasonal AutoRegressive Integrated Moving Average): Dette er den beste standardmodellen for tidsserier med klare sesongmønstre (som strømpriser har). SARIMA fanger:

Trend: Langsiktig økning/reduksjon i prisen.

Sesong: Faste mønstre som gjentar seg hvert år (f.eks. høye priser om vinteren).

Autoregresjon (AR) og Glidende Gjennomsnitt (MA): Avhengigheten mellom dagens pris og tidligere priser/feil.

Enkel Regresjon med Dummy-variabler (som din nåværende modell): Denne modellen er god for forklaring og gir et raskt estimat. Den er robust for å fange lineær trend og sesong, men den antar at prisutviklingen er helt lineær, noe som sjelden er sant for strømpriser.

Holt-Winters (Trippel Eksponensiell Utjevning): Dette er en glattingsmodell som er utmerket for å fange nivå, trend og sesongmessige komponenter. Den er enklere å implementere enn SARIMA og gir ofte gode kortsiktige prognoser.

Prophet (Utviklet av Meta/Facebook): En fleksibel modell som håndterer manglende data og outliers godt. Den er ideell for tidsserier med sterk sesongvariasjon og flere helligdager/hendelser, og krever minimalt med datavask. 
- Er det mulig å forutsi timepriser nøyaktig med enkle modeller? 
Nei, det er svært vanskelig å forutsi timepriser nøyaktig med enkle modeller.

Kompleksitet: Timepriser (spotpriser) er ekstremt volatile og påvirkes øyeblikkelig av mange faktorer:

Vær: Temperatur, vind, nedbør (påvirker produksjon og forbruk).

Tilbud og Etterspørsel: Plutselige endringer i industrielt forbruk eller feil i kraftverk.

Handel: Spekulasjon og markedssentiment.

Enkle Modellers Begrensninger: Enkle lineære modeller eller sesongmodeller (som SARIMA) er effektive for langsiktige gjennomsnitt, men de mangler evnen til å fange opp de plutselige, ikke-lineære spikene og fallene som kjennetegner timepriser.

Anbefalte Modeller for Spotpriser: For nøyaktig timeprisprediksjon kreves komplekse, multivariate modeller som:

Maskinlæring (ML): Som Random Forest eller Gradient Boosting for å håndtere mange ikke-lineære input-faktorer (som værmeldinger).

Dype Læringsnettverk (DL): Som Long Short-Term Memory (LSTM)-nettverk, som er designet spesifikt for å modellere sekvensiell data og avansert tidsavhengighet.

Timeprisprediksjon krever at du fôrer modellen med eksterne faktorer (eksterne variabler) som værmeldinger, kalenderdager og historisk produksjonsdata – noe en enkel univariate modell (kun pris) ikke kan. 
- Hvordan påvirker tidsskalaen usikkerheten i prediksjonene?  
Usikkerheten i strømprisprediksjoner henger tett sammen med hvor langt frem i tid vi ser. Jo lengre tidshorisonten er, desto større blir usikkerheten.

Usikkerhet etter Tidsskala
Kort Sikt (Timer og Dager)
På kort sikt er usikkerheten lavest. Dette er fordi vi har nesten perfekt informasjon om faktorer som påvirker prisen. Vi kjenner det meste av detaljer om strømforbruk, produksjonskapasitet og værmeldinger for de neste timene og dagene. Timeprisen styres derfor hovedsakelig av disse umiddelbare og observerbare faktorene.

Medium Sikt (Uker og Måneder)
På medium sikt begynner usikkerheten å øke. Været, som er en kritisk faktor, blir vanskeligere å forutsi nøyaktig over flere uker og måneder. Usikkerheten blir større ettersom sesongfaktorer (som vinterkulde) og langsiktige økonomiske trender får større innflytelse på prisutviklingen.

Lang Sikt (År)
På lang sikt er usikkerheten høyest. Prediksjoner som strekker seg over flere år er ekstremt usikre. Prisene påvirkes av store, uforutsigbare faktorer. Dette inkluderer politiske beslutninger, store investeringer i ny infrastruktur (som utenlandskabler), og langsiktig økonomisk vekst i samfunnet.

Hovedprinsippet: Informasjonsforfall
Når du forutsier morgendagens timepris, er feilmarginen liten fordi morgendagens vær- og produksjonsforhold er nesten helt kjent.

Når man forutsier gjennomsnittsprisen neste år, må man gjette på makroøkonomisk utvikling, endringer i vannmagasinnivåer, politisk risiko og global energipris, noe som gjør prognosen til et estimat med bredere feilmargin.
---

## Del 5: Realistisk brukerperspektiv
- Legg inn forbruksmønster:  
- Regn ut **vektet pris** basert på forbruksmønster.  
- Sammenlign med Norgespris (40 øre/kWh).


Typisk husholdning bruker mest strøm morgen (06–09) og kveld (16–22). 
Vi deler døgnet i fire perioder for å anslå når en typisk husholdning bruker strøm. Til sammen utgjør forbruket over disse 24 timene 100 prosent.

Morgenrushet (06:00 – 09:00): I disse tre timene brukes omtrent 15 prosent av strømmen. Dette forbruket skyldes ofte aktiviteter som matlaging, dusjing og generelt forbruk knyttet til påkledning og gjøremål før dagen starter.

Dagtid (09:00 – 16:00): I løpet av de syv timene midt på dagen brukes rundt 25 prosent av strømmen. Dette inkluderer strøm til hjemmekontor, lading av utstyr, og energibruk til standby på elektronikk.

Kveldsrushet (16:00 – 22:00): Kveldstimene, som varer i seks timer, er perioden med høyest forbruk, og står for hele 45 prosent av strømmen. Hovedårsakene er ofte matlaging, bruk av vaskemaskin og tørketrommel, samt økt bruk av TV og oppvarming/varmtvann i huset.

Natt (22:00 – 06:00)I de åtte timene på natten er forbruket lavt, på cirka 15 prosent. Strømmen går hovedsakelig til lading av elbiler eller utstyr, vedlikehold av varmtvann og standby på elektronikk.
**Spørsmål:**  
- Hva er den faktiske prisen når jeg bruker mest strøm?  

Den faktiske prisen når du bruker mest strøm er representert ved den vektede snittprisen.Vektet snittpris: Dette tallet er det beste estimatet for hva du faktisk betaler per kWh, fordi det regner inn de høyeste prisene fra de periodene der du har høyest forbruk (morgen- og kveldsrushet). Dette tallet er som regel høyere enn den uvektede (vanlige) gjennomsnittsprisen, siden du bruker mest strøm når strømmen er dyrest.Resultatet i koden: Se på utskriften under punkt 2: "Snittpris (VEKTET, Husholdning): [X] øre/kWh". Dette tallet (X) er din faktiske, gjennomsnittlige kostnad per kWh basert på det antatte forbruksmønsteret.Den dyreste strømmen i løpet av døgnet forekommer typisk mellom kl. 07:00–09:00 og 16:00–20:00. Ettersom ditt antatte forbruk er 15 prosent i morgenrushet og hele 45 prosent i kveldsrushet, er det disse periodenes høye timepriser som driver opp din vektede snittpris.

- Hvordan endrer dette vurderingen av Norgespris?  
Vurderingen av Norgespris endres betraktelig når vi sammenligner den med din vektede snittpris i stedet for den uvektede snittprisen.

Uten Vektet Forbruk (Uvektet)
Hvis vi bare sammenligner Norgespris (40 øre/kWh) med den enkle, uvektede historiske snittprisen (snittprisen over alle timer i døgnet), kan det se ut som Norgespris er dyrt. Dette er fordi den uvektede prisen inkluderer mange billige nattetimer vi knapt bruker strøm i.

Med Vektet Forbruk (Realistisk)
Når vi bruker den vektede snittprisen (den prisen du faktisk betaler), får vi en mer realistisk vurdering:
- Vektet Pris Nærmer Seg 40 Øre/kWh (eller går over): Hvis din vektede pris er 45 øre/kWh (som er over 40 øre/kWh), blir Norgespris et meget attraktivt forsikringsprodukt. Dette betyr at du historisk sett har betalt mer enn 40 øre/kWh i snitt for strømmen du faktisk bruker. Ved å velge Norgespris sikrer du deg en garantert rabatt på 45 øre/kWh i snitt.
- Vektet Pris Er Betydelig Under 40 Øre/kWh: Hvis din vektede pris er 35 øre/kWh (som er under 40 øre/kWh), betyr det at du selv med det høye kveldsforbruket i snitt har betalt mindre enn grensen. I dette tilfellet lønner det seg ikke å velge Norgespris, da du betaler en premie (5 øre/kWh i snitt) for å binde prisen til 40 øre/kWh.

Oppsummert: Den vektede prisen er avgjørende. Hvis ditt faktiske, historiske forbruksmønster har ført til en snittpris over 40 øre/kWh, blir Norgespris et mer fordelaktig valg enn du kanskje trodde basert på den enkle, uvektede snittprisen.
---

## Individuell leveranse
 
1. Notebook med kode, visualiseringer og analyser for alle delene
2. Kommentarer i kode; skriv kommentarer for de ulike stegne 
3. Notebook leveres på egen repo på girhub. 
4. README.md skal inneholde:
    - Kort sammendrag av hva dere har gjort.
    - En illustrasjon/eksempel på resultatet.

---
